In [29]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
import joblib

In [30]:
df = pd.read_csv("Daily Demand Forecasting Orders.csv")

In [31]:
df = df.drop(columns=["Unnamed: 0"], errors='ignore')

In [32]:
df.columns = ["week", "day", "non_urgent", "urgent", "fiscal_orders", "traffic_orders", "total_orders"]

In [33]:
X = df.drop(columns=["total_orders"])
y = df["total_orders"]

In [34]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [35]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [36]:
model = GradientBoostingRegressor(n_estimators=200, learning_rate=0.1, max_depth=5, random_state=42)
model.fit(X_train, y_train)

GradientBoostingRegressor(max_depth=5, n_estimators=200, random_state=42)

In [37]:
y_pred = model.predict(X_test)
print("MAE:", mean_absolute_error(y_test, y_pred))
print("MSE:", mean_squared_error(y_test, y_pred))
print("R2 Score:", r2_score(y_test, y_pred))

MAE: 32.90139938722745
MSE: 3135.1976094838806
R2 Score: 0.6936905419039592


In [38]:
joblib.dump(model, "demand_forecast_model.pkl")
joblib.dump(scaler, "scaler.pkl")

['scaler.pkl']

In [1]:
import streamlit as st
import pandas as pd
import joblib
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import IsolationForest

# Load trained model and scaler
model = joblib.load("demand_forecast_model.pkl")
scaler = joblib.load("scaler.pkl")

# Load dataset for visualization
df = pd.read_csv("Daily Demand Forecasting Orders.csv")
df = df.drop(columns=["Unnamed: 0"], errors='ignore')
df.columns = ["week", "day", "non_urgent", "urgent", "fiscal_orders", "traffic_orders", "total_orders"]

# Sidebar navigation
st.sidebar.title("Navigation")
page = st.sidebar.radio("Go to", ["Home", "Prediction", "Visualizations", "Insights", "Forecasting & Anomalies"])

if page == "Home":
    st.title("Daily Demand Forecasting")
    st.image("forecasting_image.jpg", use_column_width=True)
    st.write("Welcome to the Daily Demand Forecasting App. This tool helps predict total orders based on past trends and key factors such as urgent and non-urgent orders.")

elif page == "Prediction":
    st.title("Predict Total Orders")
    week = st.number_input("Week of the month", min_value=1, max_value=5, step=1)
    day = st.number_input("Day of the week", min_value=1, max_value=7, step=1)
    non_urgent = st.number_input("Non-urgent orders", min_value=0.0, step=1.0)
    urgent = st.number_input("Urgent orders", min_value=0.0, step=1.0)
    fiscal_orders = st.number_input("Fiscal sector orders", min_value=0.0, step=1.0)
    traffic_orders = st.number_input("Orders from traffic controller sector", min_value=0, step=1)
    
    if st.button("Predict Demand"):
        if non_urgent == 0 and urgent == 0 and fiscal_orders == 0 and traffic_orders == 0:
            prediction = 0
        else:
            input_data = np.array([[week, day, non_urgent, urgent, fiscal_orders, traffic_orders]])
            input_data_scaled = scaler.transform(input_data)
            prediction = model.predict(input_data_scaled)[0]
        
        st.success(f"Predicted Total Orders: {prediction:.2f}")

elif page == "Visualizations":
    st.title("Data Visualizations")
    
    st.subheader("Total Orders Over Weeks (Line Plot)")
    fig, ax = plt.subplots()
    sns.lineplot(x=df["week"], y=df["total_orders"], marker="o", ax=ax)
    ax.set_xlabel("Week")
    ax.set_ylabel("Total Orders")
    st.pyplot(fig)
    
    st.subheader("Distribution of Total Orders (Box Plot)")
    fig, ax = plt.subplots()
    sns.boxplot(x=df["total_orders"], ax=ax)
    ax.set_xlabel("Total Orders")
    st.pyplot(fig)

elif page == "Forecasting & Anomalies":
    st.title("Forecasting & Anomaly Detection")
    
    # Forecasting next few weeks' demand using moving average
    df["moving_avg"] = df["total_orders"].rolling(window=3, min_periods=1).mean()
    st.subheader("Forecasted Demand Over Next Weeks")
    fig, ax = plt.subplots()
    sns.lineplot(x=df["week"], y=df["moving_avg"], marker="o", ax=ax, label="Forecast")
    sns.lineplot(x=df["week"], y=df["total_orders"], marker="o", ax=ax, label="Actual")
    ax.set_xlabel("Week")
    ax.set_ylabel("Total Orders")
    st.pyplot(fig)
    
    # Anomaly Detection using Isolation Forest
    st.subheader("Anomaly Detection in Orders")
    iso_forest = IsolationForest(contamination=0.05, random_state=42)
    df["anomaly"] = iso_forest.fit_predict(df[["total_orders"]])
    df["anomaly"] = df["anomaly"].map({1: "Normal", -1: "Anomaly"})
    
    fig, ax = plt.subplots()
    sns.scatterplot(x=df["week"], y=df["total_orders"], hue=df["anomaly"], palette={"Normal": "blue", "Anomaly": "red"}, ax=ax)
    ax.set_xlabel("Week")
    ax.set_ylabel("Total Orders")
    st.pyplot(fig)
    
    st.write("Red points indicate anomalies in order demand trends, suggesting unusual spikes or drops.")


2025-03-01 09:13:31.463 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-01 09:13:33.794 
  command:

    streamlit run c:\Users\User\OneDrive\Desktop\daily\venv\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-03-01 09:13:33.794 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-01 09:13:33.794 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-01 09:13:33.794 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-01 09:13:33.805 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-01 09:13:33.805 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-01 09:13:33.810 Sessio